In [ ]:
from sagemaker.chainer import Chainer
from sagemaker import get_execution_role
import sagemaker
import boto3
role = get_execution_role()

bucket ="<bucketname>"

sagemaker_session = sagemaker.Session()

train_channel ='yelp.train.blazingtext.txt'
validation_channel='yelp.test.blazingtext.txt'



This launches the job on a device with no cpu.. hence slow ..

In [ ]:
s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)
prefix = 'blazingtext/supervised'
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [ ]:
region_name = boto3.Session().region_name
container = '825641698319.dkr.ecr.us-east-2.amazonaws.com/blazingtext:latest'
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

In [ ]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sagemaker_session)

In [ ]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)


In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)